In [ ]:
from lxml import html, etree
import requests
import pandas as pd
import regex
import os

In [ ]:
def get_source_code(url):
    loaded_data = requests.get(url)
    return loaded_data.text

In [55]:
url_test = 'https://www.marketwatch.com/search?q=microsoft&m=Keyword&rpp=100&mp=159&bd=false&rs=true'
news_list_test = []
extract_headlines(url_test, news_list_test)

100

In [ ]:
def extract_headlines(url, news_list):    
    text = get_source_code(url)
    tree = html.fromstring(text)
    headlines = tree.xpath('.//div[@class = "searchresult"]/a/text()')
    dates = tree.xpath('.//div[@class="deemphasized"]/span/text()')
    links = tree.xpath('.//div[@class = "searchresult"]/a/@href')
    
    for headline, date, link in zip(headlines, dates, links):
        news_list.append({
            'date' : date,
            'headline' : headline,
            'link' : link
        })
        
    return len(headlines)

In [ ]:
def load_news(url_first_page):
    news_list = []
    extract_headlines(url=url_first_page, news_list=news_list)
    print('Loaded headlines: ', len(news_list), flush=True)
    
    num = 1
    news_quantity = 100
    while news_quantity < 1000:
        current_page_url = url_first_page + '&o={}'.format(num * 100 + 1)
        headlines_num = extract_headlines(url=current_page_url, news_list=news_list)
        num += 1
        news_quantity += headlines_num
        if(headlines_num == 0):
            break
        print('Loaded headlines: ', len(news_list), flush=True)
        
    return pd.DataFrame(news_list)

In [ ]:
def load_news_by_tegs_in_csv_from_marketwatch(tegs_list):
    for teg in tegs_list:
        teg_url = 'https://www.marketwatch.com/search?q={}&m=Keyword&rpp=100&mp=159&bd=false&rs=true'.format(teg)
        print('----Loading for', teg)
        teg_pandas_dataframe = load_news(url_first_page=teg_url)
        
        if not os.path.exists("marketwatch"):
            os.makedirs("marketwatch")
        teg_pandas_dataframe.to_csv('marketwatch/{}.csv'.format(teg))
        
    print('-------Loading finished-------')

In [ ]:
tegs = ['microsoft', 'apple']
load_news_by_tegs_in_csv_from_marketwatch(tegs_list=tegs)